In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
result = requests.get("https://www.airlinequality.com/airline-reviews/british-airways")

In [3]:
type(result)

requests.models.Response

In [8]:
#result.text

In [5]:
soup = BeautifulSoup(result.text,"lxml")

In [9]:
#soup

In [7]:
soup.select('title')[0].getText()

'British Airways Customer Reviews - SKYTRAX'

In [8]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10
page_size = 100

reviews = []
for i in range(1, pages+1):
    print(f'Scraping pages {i} of 10')
    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"
    response = requests.get(url)
    
    #parsing the content
    content = response.content
    parsed_content = BeautifulSoup(content,'html.parser')
    #finding all divs and fetching the strings
    for divs in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(divs.get_text())
        
    print(f"   ---> {len(reviews)} total reviews")
    
    

Scraping pages 1 of 10
   ---> 100 total reviews
Scraping pages 2 of 10
   ---> 200 total reviews
Scraping pages 3 of 10
   ---> 300 total reviews
Scraping pages 4 of 10
   ---> 400 total reviews
Scraping pages 5 of 10
   ---> 500 total reviews
Scraping pages 6 of 10
   ---> 600 total reviews
Scraping pages 7 of 10
   ---> 700 total reviews
Scraping pages 8 of 10
   ---> 800 total reviews
Scraping pages 9 of 10
   ---> 900 total reviews
Scraping pages 10 of 10
   ---> 1000 total reviews


In [9]:
#reviews

In [10]:
import pandas as pd

In [11]:
df = pd.DataFrame()

In [12]:
df["Reviews"] = reviews

In [13]:
df.head()

,Reviews
0,✅ Trip Verified | I was pleasantly surprised ...
1,"✅ Trip Verified | British Airways is late, th..."
2,✅ Trip Verified | Flew from Amman to London on...
3,✅ Trip Verified | This is the worst experience...
4,✅ Trip Verified | Flying LHR T5 to CPT Novemb...


In [14]:
df.to_csv("Reviews.csv")

## Now we have our data set , lets work on it now

In [98]:
df= pd.read_csv("Reviews.csv")

In [99]:
df.head()

,Unnamed: 0,Reviews
0,0,✅ Trip Verified | I was pleasantly surprised ...
1,1,"✅ Trip Verified | British Airways is late, th..."
2,2,✅ Trip Verified | Flew from Amman to London on...
3,3,✅ Trip Verified | This is the worst experience...
4,4,✅ Trip Verified | Flying LHR T5 to CPT Novemb...


In [100]:
df.columns

Index(['Unnamed: 0', 'Reviews'], dtype='object')

In [101]:
df=df.drop("Unnamed: 0",axis=1)

In [102]:
df.head()

,Reviews
0,✅ Trip Verified | I was pleasantly surprised ...
1,"✅ Trip Verified | British Airways is late, th..."
2,✅ Trip Verified | Flew from Amman to London on...
3,✅ Trip Verified | This is the worst experience...
4,✅ Trip Verified | Flying LHR T5 to CPT Novemb...


In [103]:
df.isnull().sum()

Reviews    0
dtype: int64

In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Reviews  1000 non-null   object
dtypes: object(1)
memory usage: 7.9+ KB


In [10]:
#df['Reviews'].str.split('|')[993]

### Removing this row as it is not showing any information about verified or not

In [106]:
df=df.drop([993],axis=0) 

In [107]:
df=df.drop([994],axis=0)

In [108]:
df=df.reset_index()

In [109]:
df = df.drop('index',axis=1)

In [110]:
df.tail(10)

,Reviews
988,✅ Trip Verified | Worst business class I flew...
989,✅ Trip Verified | Beijing to London. Service ...
990,✅ Trip Verified | London to Beijing. Left fro...
991,✅ Trip Verified | Bangkok to London. The wors...
992,✅ Trip Verified | Food on this 4+ hours flight...
993,✅ Trip Verified | London to Belfast. Another r...
994,✅ Trip Verified | Very full flight on G-BNLP/B...
995,✅ Trip Verified | Warsaw to London. WAW is not...
996,✅ Trip Verified | I booked my flight with Cat...
997,✅ Trip Verified | Flew British Airways from Li...


In [111]:
df.head(10)

,Reviews
0,✅ Trip Verified | I was pleasantly surprised ...
1,"✅ Trip Verified | British Airways is late, th..."
2,✅ Trip Verified | Flew from Amman to London on...
3,✅ Trip Verified | This is the worst experience...
4,✅ Trip Verified | Flying LHR T5 to CPT Novemb...
5,Not Verified | Worst experience ever. Outbound...
6,✅ Trip Verified | Check in was a shambles at ...
7,✅ Trip Verified | Beyond disgusted with the fa...
8,✅ Trip Verified | On July 19th 2022 I had subm...
9,✅ Trip Verified | I booked the flight on Oct ...


In [114]:
for i in range(len(df)):
    df['Reviews'][i] = df['Reviews'].str.split('|')[i][1]

In [115]:
df.head()

,Reviews
0,I was pleasantly surprised that the airline ...
1,"British Airways is late, their website is at..."
2,Flew from Amman to London on Nov. 14 2022. No...
3,This is the worst experience I have ever had ...
4,Flying LHR T5 to CPT November 2022: BA app ...


In [116]:
df.to_csv("Reviews.csv")

# TRYING WITH NLTK

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")

import nltk

In [3]:
df = pd.read_csv('Reviews.csv')

In [5]:
df=df.drop('Unnamed: 0',axis=1)

In [6]:
df.head()

,Reviews
0,I was pleasantly surprised that the airline ...
1,"British Airways is late, their website is at..."
2,Flew from Amman to London on Nov. 14 2022. No...
3,This is the worst experience I have ever had ...
4,Flying LHR T5 to CPT November 2022: BA app ...


In [7]:
df.shape

(998, 1)